In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import google.colab

google.colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyarrow
!pip install snappy
!pip install fastparquet
!pip install python-snappy
!sudo apt-get install libsnappy-dev

     |████████████████████████████████| 26.5MB 166kB/s 
     |████████████████████████████████| 655kB 38.6MB/s 
     |████████████████████████████████| 45.0MB 92kB/s 
     |████████████████████████████████| 727kB 33.5MB/s 
     |████████████████████████████████| 655kB 34.8MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 10.0MB 13.5MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp36-none-any.whl size=67162 sha256=ed860911e78e4d5678a7986cafe7ab90be1c3ea1a78e653aa7fa384c8cd77743
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d4722c2ae9486691b1d49b922b9877922beeabb3
Successfully built pypng
  Using cached https://files.pythonhosted.org/packages/45/35/65d9f8cc537129894b4b32647d80212d1fa342877581c5b8a69872cea8be/python-snappy-0.5.4.tar.gz
  Created wheel for python-snappy: filename=python_snappy-0.5.4-cp36-cp36m-linux_x86_64.whl size=40387 sha256=4b4e542ac48509d7ab48ccf131c75818cb4529b1fb14e9ffab0f0f59718105ab
  St

In [3]:
DATADIR = "/content/drive/My Drive/lenta_data"
df = pd.read_parquet(f"{DATADIR}/transactions.parquet", engine='fastparquet', columns=["chq_id", "chq_date", "client_id", "sales_sum", "sales_count"])
df.sample(5)

,chq_id,chq_date,client_id,sales_sum,sales_count
16407942,271bf8cc3b60a849f0b9f55b639aac47,2017-03-28,f4cb6d98c26f0d039cdb18a6b5508181,124.07,1.034
18034279,75e3362fb91f17e30cc65fe998486c10,2017-05-20,9ce489b816256a866115a739d87e61e8,77.58,2.000
2413782,b955a1d39201a5fae680ef2346618786,2016-11-20,11949d1d4dbb67a8f4a45b62f03226df,118.99,1.000
19704572,d77c7687036722fe836bec01ca22bd67,2017-05-28,5b6ac053122cd02888fb5fca80403db9,73.98,2.000
6668931,6deade7361b9fecd2f0434690cd46f03,2016-12-24,75f764c863dcb54dcb842f023243880d,214.44,1.000


In [4]:
print("Number of entries:", len(df))
df.info(verbose=False, memory_usage='deep')

Number of entries: 32109414
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32109414 entries, 0 to 32109413
Columns: 5 entries, chq_id to sales_count
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 6.0 GB


In [ ]:
# убрать возвраты
df = df[df['sales_count'] > 0]
df.info(verbose=False, memory_usage='deep')

In [7]:
print("Number of entries:", len(df))

Number of entries: 32086641


In [9]:
client_info = df.groupby(['client_id'])['chq_id'].nunique().to_frame()
client_info.rename(columns={'chq_id' : 'n_chqs'}, inplace=True)
client_info.sample(5)

,n_chqs
client_id,
5fd1f14231248e1cc832f4b4100fe2b8,79
9e3cd320b13ecc4de738b4a086d78846,10
89d9c6ac404fb99e16709a96bc7a40c9,132
4982b51e88342a8d3b1695f67ea26de2,36
e6d30046717d0f2955abca504e0013d0,39


In [11]:
df.drop(columns=["chq_id"], inplace=True)
df.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32086641 entries, 0 to 32109413
Columns: 4 entries, chq_date to sales_count
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 4.9 GB


In [13]:
client_info['mean_chq_sum'] = df.groupby(['client_id'])['sales_sum'].sum() / client_info['n_chqs']
client_info.sample(5)

,n_chqs,mean_chq_sum
client_id,,
ba4b79c697d5c220b7279fd227fae82b,5,219.800000
0f9bd407125c4b163197116b814d0e9e,29,875.758621
b6f05ff3abd23b5c11811ca3a69722db,91,1384.384615
1f8891be8c3d265c5e5b4b570a1c1fdd,25,6751.520000
582ace875e852ec895560ab6c8c6244e,56,1909.464464


In [14]:
df.drop(columns=["sales_sum", "sales_count"], inplace=True)
df.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32086641 entries, 0 to 32109413
Columns: 2 entries, chq_date to client_id
dtypes: datetime64[ns](1), object(1)
memory usage: 4.4 GB


In [15]:
client_info['last_purchase'] = df.groupby(['client_id'])['chq_date'].max()
client_info.sample(5)

,n_chqs,mean_chq_sum,last_purchase
client_id,,,
98de13e2a1685294b9d8ab92220e00d6,75,1103.826667,2017-09-20
cdb42a1a82d29f10426bf388fceaf8c7,56,259.071607,2017-09-15
620e46702b8511a956c539f48feafef0,7,5150.857143,2017-06-01
059bfb36251b289b199187f8aa30f502,95,325.831579,2017-10-04
339e4077ed1303c638d6468cd23710a5,40,1253.175000,2017-10-03


In [22]:
from collections import defaultdict

statistics = defaultdict(list)
cols = ['n_chqs', 'mean_chq_sum', 'last_purchase']

for col in cols:
    statistics[col].append(client_info[col].quantile(0.33))
    statistics[col].append(client_info[col].quantile(0.66))
statistics

defaultdict(list,
            {'last_purchase': [Timestamp('2017-08-07 00:00:00'),
              Timestamp('2017-09-25 00:00:00')],
             'mean_chq_sum': [852.4804102564112, 1578.947142857145],
             'n_chqs': [12.0, 29.0]})

In [48]:
def get_weight(x, statistics):
    weight = 0
    cols = ['n_chqs', 'mean_chq_sum', 'last_purchase']
    importances = [0.3, 0.3, 0.4]

    for col, importance in zip(cols, importances):
        if x[col] > statistics[col][1]:
            weight += importance
        elif x[col] < statistics[col][0]:
            weight += 0
        else:
            weight += importance * 0.5
    return round(weight, 2)

In [49]:
client_info['weight'] = client_info.apply(lambda x: get_weight(x, statistics), axis=1)
client_info.sample(5)

,n_chqs,mean_chq_sum,last_purchase,weight
client_id,,,,
e2b559f8cbd2d24a3535a5c2998bcbba,29,902.663448,2017-09-29,0.70
ee604b330c4dabd20106f30fdcb4c709,100,479.770000,2017-07-23,0.30
84ae84fd7f127a4f693cfc23e38a022e,89,1949.112360,2017-10-03,1.00
52cac54010f1e269e057154504677e8e,23,2177.869565,2017-06-17,0.45
c40e62d3931a54f1c373d0f9098aeb3d,6,2066.166667,2017-03-28,0.30


In [51]:
client_info.reset_index(level=0, inplace=True)
client_info.sample(5)

,client_id,n_chqs,mean_chq_sum,last_purchase,weight
13675,2301f14b978a1529932cdcc7f0c8cbcb,18,1847.222222,2017-08-22,0.65
55396,8e600be3f7c4a1470b46ac479b7dd1b7,136,527.904485,2017-10-04,0.70
69046,b13f6a7238ede7c589c48790abbf3932,4,2613.750000,2016-11-20,0.30
14336,24bc205d5c57fdd9b669d61093c4e751,30,1276.533333,2017-09-24,0.65
29926,4d02c88a0f885797ae782f4ee2f10f02,47,725.703191,2017-09-13,0.50


In [59]:
client_info.drop(columns=['Unnamed: 0'], inplace=True)
client_info.to_csv(f'{DATADIR}/client_weights.csv', index=False)

In [60]:
# check
client_info = pd.read_csv(f'{DATADIR}/client_weights.csv')
client_info.sample(5)

,client_id,n_chqs,mean_chq_sum,last_purchase,weight
99520,fed145ecdc307fbee8a5ff45ae211b6b,15,1688.600000,2017-09-05,0.65
50701,8259f7ad2a8e5b55bd115514677adae8,26,732.807692,2017-08-27,0.35
18306,2ee7563be4dad85a32af8990840c0c3a,7,315.285714,2017-08-29,0.20
43930,70e64150bc57907c8ffb920c272da9ce,7,1359.000000,2017-04-01,0.15
63067,a1da6b4c57e4ee28d8773e13526c6973,5,725.000000,2017-03-29,0.00
